In [1]:
import pandas as pd
import re

In [15]:
df = pd.read_csv("precios.csv")

In [16]:
df.dtypes

medicamento             object
descripcion             object
peso                    object
presentacion           float64
forma_farmacologica     object
marca                   object
precio                  object
max_price               object
precio descontado      float64
descuento               object
promotion               object
fuente                  object
scrapping_day           object
detail_url              object
dtype: object

In [17]:
df.shape

(31289, 14)

In [21]:
df = df[df['scrapping_day']=="11/10/2024"].copy()
df.shape

(2177, 14)

In [ ]:
df.sample(3)

# formas farmacologicas

In [22]:
def normalizar(text: str) -> str:
    """retorna la cadena del parametro sin acentos, en minuscula 
       y quita espacios al principio o fin de la cadena """
       
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    try:
        text_normalized = text.translate(trans).lower().strip()
    except:
        print(f"error normalizando: {text}")
        text_normalized = "Error"
    return text_normalized

def get_concentracion_from_description(desc_comer) -> list:
   """ get the four characteristics from descripcion comercial( eg. 20 mg 20 tabletas)
    receives an import row from matched_imports.csv, tries to find the 
    presentation(concentration, concentration unit, presentation, farmacology form) 
    from the comercial description("descripcion_comercial") through differents forms
    with regular expressions.

    Returns
    -------
    list-like: 
         a list with concentration, concentration unit, presentation, farmacology form
         eg. 150 mg 20 tabletas which means: 150 milligrams 20 tablets
        
    explicacion extendida:
    ----------------------
        El import tiene una descripcion comercial como esta:
           # "ninlaro (ixazomib) caja colectiva con 3 cajas con una capsula de 3.5 mg (uso humano)"
        La sigueinte expresion regular encuentra la parte de la cadena que indica las 4 caracteristicas
        de su presentacion
           # \d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion oral|ge)
        En este caso devuelve esto: 3 cajas con una capsula de 3.5 mg 
        que se convierte en: 3 capsulas 3.5 mg
        Las regex descritas abajo se conforman básicamente de 4 partes que se alternan entre sí para
        resolver las multiples patrones encontrados en su descripcion comercial
        1) concentracion -> puede ser numero o flotante
           # regex:  \d+\.?\d*\s* numero entero o decimal seguido de 0,1 o más espacios
        2) unidad concentracion -> siempre mg por ahora 
           # mg
        3) presentacion
           # \d{1,3} número de capsulas, siempre es entero y debe existir minimo una vez y maximo 3
        4) forma farmacologica
            #(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral|ge) tiene que haber por 
            #lo menos una de estas formas
    """
   desc_comer = normalizar(desc_comer)
   #------------------------------4------------------------------#
   unidad_concentracion = '(mg|mcg|ml|gr)'
   cant_flot = '\d+\.?\d*\s*'
   cant_ent = '\d{1,3}'
   forma_farma = '(comprimidos|ampolletas|inyectable|suspension|tabletas|capsulas|solucion|jeringas|capsula|aerosol|parches|sobre|polvo|dosis|vial|caps|tabl|amp|tab|gel|ud)'
   espacios = '\s*'
   any = '.{0,40}'
   # 'seretide diskus polvo 50mcg/500mcg, 60 dosis.'
   expresion_a = cant_flot+unidad_concentracion + "/" + cant_flot + unidad_concentracion + any + cant_ent + espacios + forma_farma
   expresion_b =  cant_flot + espacios + unidad_concentracion  + any + cant_ent + espacios + forma_farma
   expresion_c = cant_ent + espacios + forma_farma + any + cant_flot + unidad_concentracion
   expresion_d = forma_farma + any + cant_flot + espacios + unidad_concentracion
   expresion_e = cant_flot + espacios + unidad_concentracion  + any + forma_farma
   expresion_f = cant_ent + espacios + forma_farma
   expresion_g = cant_flot + espacios + unidad_concentracion
   expresion_h = cant_flot + espacios + "g" 

   if not isinstance(desc_comer, str):
       return ["", "",""]
   #----------------------------------------4--------------------------------------#
   #seretide diskus polvo 50mcg/500mcg, 60 dosis.' -4
   match = re.search(expresion_a, desc_comer)
   if match:
      info = match.group()

      regex = cant_flot+unidad_concentracion + "/" + cant_flot + unidad_concentracion
      peso = re.search(regex, info)
      peso = peso[0]

      regex = cant_ent + espacios + forma_farma
      cantidad_forma_farma = re.search(regex, info)
      cantidad = cantidad_forma_farma[0].split(" ")[0]
      forma_farmacologica = cantidad_forma_farma[0].split(" ")[1]
      
      return [peso, cantidad, forma_farmacologica]
 
   # 'kitoscell lp 600 mg, 90 tabletas.' -4
   match = re.search(expresion_b, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion
      peso = re.search(regex, info)
      peso = peso[0]

      regex = cant_ent + espacios + forma_farma
      cantidad_forma_farma = re.search(regex, info)
      cantidad = cantidad_forma_farma[0].split(" ")[0]
      forma_farmacologica = cantidad_forma_farma[0].split(" ")[1]
      peso = peso.replace(" ", "")
      return [peso, cantidad, forma_farmacologica]

   # 
   match = re.search(expresion_c, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion
      peso = re.search(regex, info)
      peso = peso[0]

      regex = cant_ent + espacios + forma_farma
      cantidad_forma_farma = re.search(regex, info)
      cantidad = cantidad_forma_farma[0].split(" ")[0]
      forma_farmacologica = cantidad_forma_farma[0].split(" ")[1]
      
      peso = peso.replace(" ", "")
      return [peso, cantidad, forma_farmacologica]

   # 'fosfocil 1 gr solución inyectable intravenosa, 1 pz.' -3 o -4
   match = re.search(expresion_e, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion 
      peso = re.search(regex, info)
      peso = peso[0]

      regex = forma_farma
      forma_farmacologica = re.search(regex, info)[0]

      cantidad = ''
      match = re.search("\d{1,3}\s*(pz|pzas|piezas)", desc_comer)
      if match:
         cantidad = re.search("\d{1,3}", info)[0] 
         
      return [peso, cantidad, forma_farmacologica]
 
   # 'kitoscell gel, 30 gr.' -3 
   match = re.search(expresion_d, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion
      peso = re.search(regex, info)
      peso = peso[0]

      regex = forma_farma
      forma_farmacologica = re.search(regex, info)[0]
      peso = peso.replace(" ", "")
      return [peso, "", forma_farmacologica]

   # 'allegra d tratamiento para la alergia y congestion nasal antihistaminico, 10 tabletas.' -2
   match = re.search(expresion_f, desc_comer)
   if match:
      info = match.group()
      regex = cant_ent + espacios + forma_farma
      cantidad_forma_farma = re.search(regex, info)
      cantidad = cantidad_forma_farma[0].split(" ")[0]
      forma_farmacologica = cantidad_forma_farma[0].split(" ")[1]
      return ["", cantidad, forma_farmacologica]
 
   #'atrovent 250mcg/ml, 20 ml.' -2
   match = re.search(expresion_g, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion
      peso = re.search(regex, info)
      peso = peso[0]
      peso = peso.replace(" ", "")
      return [peso, "", ""]

   # Kitoscell Gel 10 g
   match = re.search(expresion_h, desc_comer)
   if match:
      peso = match.group()
      peso = peso.replace(" ", "")
      return [peso, "", ""]
   
   return ["", "",""]

In [23]:
def format_df(dfs):
    peso, presentacion, forma_farmacologica = get_concentracion_from_description(dfs['descripcion'])
    
    return (peso, presentacion, forma_farmacologica)

#df.apply(format_df, axis=1)

In [24]:
df[['peso', 'presentacion', 'forma_farmacologica']] = df.apply(format_df, axis=1, result_type='expand').copy()
df.sample(1)

,medicamento,descripcion,peso,presentacion,forma_farmacologica,marca,precio,max_price,precio descontado,descuento,promotion,fuente,scrapping_day,detail_url
29346,arcoxia,Arcoxia 28 Comprimidos Caja Etoricoxib 60 MG,60mg,28,comprimidos,NaN,1808.0,NaN,NaN,NaN,Programa Club Salud: Acumula 3 y el 4º es GRATIS.,Farmacias San Pablo,11/10/2024,/medicamentos/supervision-medica/a---b---c/arc...


In [ ]:
df[(df['peso'] != '') & (df['presentacion'] != '') & (df['forma_farmacologica'] != '')].shape

In [56]:
df[['descripcion', 'peso', 'presentacion', 'forma_farmacologica']].to_csv("pb1.csv")

# medicamento not in descripcion

In [13]:
mask = df.apply(lambda x: True if x.medicamento not in x.descripcion.lower() else False, axis=1)

In [ ]:
df[mask][['medicamento', 'descripcion', 'fuente']].shape

In [ ]:
df[(df['medicamento']=='galvus')]

In [ ]:
key_medicines = [
    'GIOTRIF', 'TARCEVA', 'IRESSA', 'VARGATEF', 'TAXOTERE', 'KEYTRUDA', 'OPDIVO', 'ACTILYSE',
    'METALYSE', 'VARIDASA', 'OFEV', 'KITOSCELL', 'SERETIDE', 'PULMICORT', 'ZENHALE', 'SYMBICORT', 
    'VANNAIR', 'RELVARE', 'COMBIVENT', 'VENTOLIN', 'STERIVENT', 'ATROVENT', 'ALUPENT', 'VINZA',
    'SPIRIVA', 'STRIVERDI', 'ONBRIZE', 'SEEBRI', 'EKLIRA', 'OSLIF', 'SPIOLTO', 'ULTIBRO', 'ANORO', 
    'DUAKLIR', 'ULUNAR', 'TRIMBOW', 'TRELEGY', 'MACRODANTINA', 'FURADANTINA', 'MICTASOL', 'FOSFOCIL',
    'VODELAN', 'NITROFURANT.GI', 'BACTRIM', 'SECOTEX', 'XATRAL-OD', 'AVODART', 'COMBODART', 'PROSCAR', 
    'ASOFLON', 'PROFIDOX', 'UPROSOL', 'TAMSULOSINA', 'SIFROL', 'AZILECT', 'NUBRENZA', 'STALEVO', 
    'SINEMET', 'SUNAM', 'CLOISONE', 'FLURINOL', 'AVIANT', 'XUZAL', 'ALLEGRA', 'VIRLIX', 'MOBICOX', 
    'VOLTAREN', 'EXEL', 'CELEBREX', 'MELOSTERAL', 'ARCOXIA', 'TRAYENTA', 'JANUVIA', 'ONGLYZA', 'GALVUS', 
    'FAZIQUE', 'INCRESINA', 'XILIARXS', 'JALRA', 'JANUMET', 'KOMBIGLYZE', 'VELMETIA', 
    'JARDIANZ', 'XIGDUO', 'INVOKANA', 'TRULICITY', 'OZEMPIC', 'CATAPRESAN', 'CARDURA', 'MICARDIS', 
    'EDARBI', 'LOSARTAN', 'ATACAND', 'ALMETEC', 'APROVEL', 'AVAPRO', 'COZAAR', 'DIOVAN', 'LEGIONIS', 
    'TELARTEQ', 'TRANSENDIS', 'AVALIDE', 'HYZAAR', 'COAPROVEL', 
    'LOSAR', 'OPENVAS', 'EXFORGE', 'APROVASC', 'BICARTIAL', 'DUOALMETEC', 'MAXOPRESS', 
    'PRADAXAR', 'XARELTO', 'ELICUIS', 'CLEXANE', 'COUMADIN', 'SINTROM', 'FRAXIPARINE'
]
len(key_medicines)

# palabras de busqueda

In [7]:
def normalizar(text: str) -> str:
    """retorna la cadena del parametro sin acentos, en minuscula 
       y quita espacios al principio o fin de la cadena """
       
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    try:
        text_normalized = text.translate(trans).lower().strip()
    except:
        print(f"error normalizando: {text}")
        text_normalized = "Error"
    return text_normalized


def get_concentracion_from_description(desc_comer) -> list:
   """ get the four characteristics from descripcion comercial( eg. 20 mg 20 tabletas)
    receives an import row from matched_imports.csv, tries to find the 
    presentation(concentration, concentration unit, presentation, farmacology form) 
    from the comercial description("descripcion_comercial") through differents forms
    with regular expressions.

    Returns
    -------
    list-like: 
         a list with concentration, concentration unit, presentation, farmacology form
         eg. 150 mg 20 tabletas which means: 150 milligrams 20 tablets
        
    explicacion extendida:
    ----------------------
        El import tiene una descripcion comercial como esta:
           # "ninlaro (ixazomib) caja colectiva con 3 cajas con una capsula de 3.5 mg (uso humano)"
        La sigueinte expresion regular encuentra la parte de la cadena que indica las 4 caracteristicas
        de su presentacion
           # \d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion oral|ge)
        En este caso devuelve esto: 3 cajas con una capsula de 3.5 mg 
        que se convierte en: 3 capsulas 3.5 mg
        Las regex descritas abajo se conforman básicamente de 4 partes que se alternan entre sí para
        resolver las multiples patrones encontrados en su descripcion comercial
        1) concentracion -> puede ser numero o flotante
           # regex:  \d+\.?\d*\s* numero entero o decimal seguido de 0,1 o más espacios
        2) unidad concentracion -> siempre mg por ahora 
           # mg
        3) presentacion
           # \d{1,3} número de capsulas, siempre es entero y debe existir minimo una vez y maximo 3
        4) forma farmacologica
            #(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral|ge) tiene que haber por 
            #lo menos una de estas formas
    """
   desc_comer = normalizar(desc_comer)
   #------------------------------4------------------------------#
   unidad_concentracion = '(mg|mcg|ml|gr|y|g)'
   cant_flot = '\d+\.?\d*\s*'
   cant_ent = '\d{1,3}'
   cant_ff = 'x \d{1,3}'
   forma_farma = '(comprimidos|ampolletas|inyectable|suspension|tabletas|capsulas|solucion|jeringas|capsula|aerosol|parches|sobre|polvo|dosis|spray|vial|caps|tabl|amp|tab|gel)'
   espacios = '\s*'
   any = '.{0,40}'
   expresion_a = forma_farma + any + cant_flot + espacios + unidad_concentracion + espacios + cant_ff

   expresion_b = cant_flot + espacios + unidad_concentracion + espacios + cant_ff
   expresion_c = forma_farma + any + cant_ff
   
   if not isinstance(desc_comer, str):
       return ["", "",""]
   
   # GIOTRIF TABL F.COATE 20 MG x 30
   match = re.search(expresion_a, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion
      peso = re.search(regex, info)
      p = re.search(cant_flot,peso[0])[0]
      try:
         int(p)
         pe = p
      except:
         pe = str(float(p))
      so = re.search(unidad_concentracion,peso[0])[0]
      peso = pe+so
      regex = cant_ff
      cant = re.search(regex, info)[0]
      cant = re.search(cant_ent, cant)[0]

      regex = forma_farma
      forma_farmacologica = re.search(regex, info)[0]
      peso = peso.replace(" ", "").replace("y", "mcg")
      return [peso, cant, forma_farmacologica]

   # ACTILYSE V.INF C/SOLV 50 MG x 2
   match = re.search(expresion_b, desc_comer)
   if match:
      info = match.group()
      regex = cant_flot + espacios + unidad_concentracion
      peso = re.search(regex, info)
      p = re.search(cant_flot,peso[0])[0]
      try:
         int(p)
         pe = p
      except:
         pe = str(float(p))
      so = re.search(unidad_concentracion,peso[0])[0]
      peso = pe+so
      regex = cant_ff
      cant = re.search(regex, info)[0]
      cant = re.search(cant_ent, cant)[0]
      peso = peso.replace(" ", "").replace("y", "mcg")
      return [peso, cant, ""]
   
   # VARIDASA TABL  ORAL x 24
   match = re.search(expresion_c, desc_comer)
   if match:
      info = match.group()
      forma_farmacologica = re.search(forma_farma, info)[0]

      regex = cant_ff
      cant = re.search(regex, info)[0]
      cant = re.search(cant_ent, cant)[0]
      
      return ["", cant, forma_farmacologica]
   
   return ["", "",""]


In [8]:
dfm = pd.read_excel('Palabras de búsqueda.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
dfm.shape

In [ ]:
dfm.sample(5)

In [16]:
def format_df(med):
    name = med['Medicamento'].strip().split(" ",1)[0].strip().lower()
    peso, presentacion, forma_farmacologica = get_concentracion_from_description(med['Medicamento'])
    
    return (name, peso, presentacion, forma_farmacologica)

#dfm.apply(format_df, axis=1)

In [ ]:
dfm[['name', 'peso', 'presentacion', 'forma_farmacologica']] = dfm.apply(format_df, axis=1, result_type='expand')
dfm.sample(7)

In [ ]:
dfm.shape

In [ ]:
dfm[(dfm['peso'] == '') & (dfm['presentacion'] == '') & (dfm['forma_farmacologica'] == '')].shape

In [19]:
dfm.to_csv("medicamentos_base_homologada.csv")

# Cross

In [25]:
dfm  = pd.read_csv('medicamentos_base_homologada.csv')

In [26]:
df = df[['medicamento','descripcion', 'peso', 'presentacion', 'forma_farmacologica']].copy()
df.sample(2)

,medicamento,descripcion,peso,presentacion,forma_farmacologica
28275,almetec,"almetec tri 40mg/5mg/12.5mg, 14 tabletas.",40mg/5mg,14,tabletas
28353,losar,"papilla heinz de mango, a partir de los 6 mese...",113gr,,


In [27]:
dfm[['Medicamento','name','peso','presentacion','forma_farmacologica']].sample(2)

,Medicamento,name,peso,presentacion,forma_farmacologica
47,ONBRIZE CAPS B.HAL 300 Y x 30,onbrize,300mcg,30.0,caps
50,SERETIDE AER.CD EVOH 125 Y x 120 (/25),seretide,125mcg,120.0,NaN


In [28]:
def homologar(df):
    med = df['medicamento']
    peso = df['peso']
    pres =df['presentacion']
    ff = df['forma_farmacologica']

    if not peso or not pres:
        if peso:
            sintetic_homolog = med + " " + ff + " " + peso
            return sintetic_homolog.replace("dosis", "").replace("mcg", "Y").strip().upper()
        
        else:
            return ""
    
    df_c = dfm[ (dfm['name']==med) & (dfm['peso']==peso) & (dfm['presentacion']==pres) ]
    coinc = len(df_c)
    if coinc > 0:
        return str(df_c['Medicamento'].iloc[0]).strip()
    
    sintetic_homolog = med + " " + ff + " " + peso + " x " + pres
    return sintetic_homolog.replace("dosis", "").replace("mcg", "Y").strip().upper()

 #df.apply(homologar, axis=1)

In [29]:
df['h'] = df.apply(homologar, axis=1)

In [30]:
df['h'].value_counts()

h
                              164
JALRA  120ML                   23
LOSAR  113GR                   12
LOSAR  85GR                    11
JALRA  250ML                   10
                             ... 
LOSAR  750ML                    1
LOSAR  225GR                    1
LOSAR  42GR                     1
LOSAR  14ML                     1
COUMADIN TABLETAS 2MG X 21      1
Name: count, Length: 1288, dtype: int64

In [68]:
df.to_csv("pb1.csv")

In [31]:
df[df['h'] != ''].shape

(2013, 6)